# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [2]:
import os

# Set your preferred name
USER = "Shuyi"
# Set the data and output directories
DATA_ROOT = '/content/drive/MyDrive/AI4Science_Public/data'
OUTPUT_ROOT = '/content/drive/MyDrive/AI4Science_Public/output1'
TRAIT = "Sjögren's Syndrome"

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [1]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')
!pip install mygene
!pip install sparse_lmm

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [4]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['.DS_Store',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Kidney_Chromophobe_(KICH)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Thymoma_(THYM)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [ ]:
trait_subdir = "TCGA_Breast_Cancer_(BRCA)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_BRCA_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_HiSeqV2_PANCAN.gz')

In [ ]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['AJCC_Stage_nature2012',
 'Age_at_Initial_Pathologic_Diagnosis_nature2012',
 'CN_Clusters_nature2012',
 'Converted_Stage_nature2012',
 'Days_to_Date_of_Last_Contact_nature2012',
 'Days_to_date_of_Death_nature2012',
 'ER_Status_nature2012',
 'Gender_nature2012',
 'HER2_Final_Status_nature2012',
 'Integrated_Clusters_no_exp__nature2012']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['AJCC_Stage_nature2012', 'Age_at_Initial_Pathologic_Diagnosis_nature2012', 'CN_Clusters_nature2012', 'Converted_Stage_nature2012', 'Days_to_Date_of_Last_Contact_nature2012', 'Days_to_date_of_Death_nature2012', 'ER_Status_nature2012', 'Gender_nature2012', 'HER2_Final_Status_nature2012', 'Integrated_Clusters_no_exp__nature2012', 'Integrated_Clusters_unsup_exp__nature2012', 'Integrated_Clusters_with_PAM50__nature2012', 'Metastasis_Coded_nature2012', 'Metastasis_nature2012', '

In [ ]:
candidate_age_cols = ['Age_at_Initial_Pathologic_Diagnosis_nature2012', 'age_at_initial_pathologic_diagnosis',
                      'days_to_birth', 'year_of_initial_pathologic_diagnosis']
candidate_gender_cols = ['Gender_nature2012', 'gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'Age_at_Initial_Pathologic_Diagnosis_nature2012': [nan, nan, nan, nan, nan],
 'age_at_initial_pathologic_diagnosis': [55.0, 50.0, 62.0, 52.0, 50.0],
 'days_to_birth': [-20211.0, -18538.0, -22848.0, -19074.0, -18371.0],
 'year_of_initial_pathologic_diagnosis': [2004.0,
  2003.0,
  2011.0,
  2011.0,
  2013.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'Gender_nature2012': [nan, nan, nan, nan, nan],
 'gender': ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Breast Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-3C-AAAU-01,1,55.0,0.0,0.607308,-3.194126,-0.531035,-0.145872,0.237422,-0.29921,-0.142694,...,-0.748878,0.565583,-0.767233,-1.28139,-0.271377,-0.492286,0.360373,1.067905,0.076267,-0.392212
TCGA-3C-AALI-01,1,50.0,0.0,-0.641192,-4.928226,0.095465,0.098128,-0.541978,-0.32291,-0.044694,...,0.312922,0.079683,0.475267,1.99661,0.161423,0.273714,0.644673,-0.428695,0.068667,-0.043812
TCGA-3C-AALJ-01,1,62.0,0.0,1.082808,-4.623726,-0.531035,0.484028,-0.183678,-0.91901,0.261106,...,-0.748878,-0.656117,-0.216733,0.21081,0.101023,0.660514,1.295073,0.915105,0.168567,0.047788
TCGA-3C-AALK-01,1,52.0,0.0,0.121608,-2.881526,-0.531035,0.179128,0.039222,-0.45491,0.180306,...,0.415422,-0.178417,-0.211233,-1.28139,0.108023,1.132814,0.496773,0.240105,3.099767,0.112888
TCGA-4H-AAAK-01,1,50.0,0.0,0.420208,-3.282726,-0.531035,-0.020972,-0.117978,-0.55781,-0.173794,...,0.685222,-0.698717,-0.484233,-1.28139,-0.223577,1.222714,0.225573,-0.301995,0.200067,-0.080212


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1215 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Breast Cancer', the least common label is '0' with 113 occurrences. This represents 9.30% of the dataset.
The distribution of the feature 'Breast Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 48.0
  50% (Median): 58.0
  75%: 67.0
Min: 26.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 13 occurrences. This represents 1.07% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [ ]:
dataset = 'GEO'
trait_subdir = "Sjögren's-Syndrome"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE66795',
 'GSE84844',
 'GSE51092',
 'GSE93683',
 'GSE75679',
 'GSE94510',
 'GSE140161',
 'GSE143153',
 'GSE135809',
 'GSE110007',
 'GSE166373',
 'GSE40611']

Repeat the below steps for all the accession numbers

In [ ]:
cohort = accession_num = "GSE40611"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

("/content/drive/MyDrive/AI4Science_Public/data/GEO/Sjögren's-Syndrome/GSE40611/GSE40611_family.soft.gz",
 "/content/drive/MyDrive/AI4Science_Public/data/GEO/Sjögren's-Syndrome/GSE40611/GSE40611_series_matrix.txt.gz")

### Inital filtering and clinical data preprocessing

In [ ]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Gene expression data of parotid tissue from Primary Sjogren’s Syndrome and controls"
!Series_summary	"Primary Sjögren’s syndrome (pSS) is a chronic autoimmune disease with complex etiopathogenesis. Here we use Affymetrix U133 plus 2.0 microarray gene expression data from human parotid tissue. Parotid gland tissues were harvested from 17 pSS and 14 14 non-pSS sicca patients and 18 controls. The data were used in the following article:  Nazmul-Hossain ANM, Pollard RPE, Kroese FGM, Vissink A, Kallenberg CGM, Spijkervet FKL, Bootsma H, Michie SA, Gorr SU, Peck AB, Cai C, Zhou H, Horvath S, Wong DTW (2012) Systems Analysis of Primary Sjögren’s Syndrome Pathogenesis in Salivary Glands: Comparative Pathways and Molecular Events in Humans and a Mouse Model."
!Series_overall_design	"Parotid gland tissues were harvested from 17 pSS and 14 non-pSS sicca patients and 18 controls."


In [ ]:
clinical_data.head()

,!Sample_geo_accession,GSM997850,GSM997851,GSM997852,GSM997853,GSM997854,GSM997855,GSM997856,GSM997857,GSM997858,...,GSM997889,GSM997890,GSM997891,GSM997892,GSM997893,GSM997894,GSM997895,GSM997896,GSM997897,GSM997898
0,!Sample_characteristics_ch1,disease status: Control,disease status: Control,disease status: Control,disease status: Control,disease status: Control,disease status: Control,disease status: Control,disease status: Control,disease status: Control,...,disease status: Sicca,disease status: Sicca,disease status: Sicca,disease status: pSS,disease status: pSS,disease status: pSS,disease status: pSS,disease status: pSS,disease status: pSS,disease status: pSS
1,!Sample_characteristics_ch1,batch: 1,batch: 1,batch: 1,batch: 1,batch: 1,batch: 1,batch: 1,batch: 1,batch: 1,...,batch: 3,batch: 3,batch: 3,batch: 2,batch: 2,batch: 2,batch: 2,batch: 3,batch: 3,batch: 3


In [ ]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['disease status: Control',
  'disease status: pSS',
  'disease status: Sicca'],
 1: ['batch: 1', 'batch: 2', 'batch: 3']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [ ]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Sjögren\'s Syndrome\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Sjögren\'s Syndrome\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. 

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [ ]:
is_gene_availabe = True
trait_row_id = 0
age_row_id = None
gender_row_id = None

trait_type = 'binary'

In [ ]:
is_available = is_gene_availabe and (trait_row_id is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [ ]:
# Verify and use the functions generated by GPT

def convert_trait(trait_value):
    # For Sjögren's Syndrome (binary)
    if trait_value == 'disease status: Control':
        return 0
    elif trait_value == 'disease status: pSS' :
        return 1
    elif trait_value == 'disease status: Sicca':
        return 0
    else:
        return None

def convert_age(age_value):
    # Assuming age is a continuous variable
    try:
        return float(age_value)
    except ValueError:
        return None

def convert_gender(gender_value):
    # For gender (binary)
    if gender_value == 'gender: Female':
        return 1
    elif gender_value == 'gender: Female':
        return 0
    else:
        return None




In [ ]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row_id, age_row_id, gender_row_id,
                                                      convert_trait, convert_age, convert_gender)
selected_clinical_data.head()

,GSM997850,GSM997851,GSM997852,GSM997853,GSM997854,GSM997855,GSM997856,GSM997857,GSM997858,GSM997859,...,GSM997889,GSM997890,GSM997891,GSM997892,GSM997893,GSM997894,GSM997895,GSM997896,GSM997897,GSM997898
Sjögren's Syndrome,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,1,1,1,1


### Genetic data preprocessing and final filtering

In [ ]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,ID,GSM997850,GSM997851,GSM997852,GSM997853,GSM997854,GSM997855,GSM997856,GSM997857,GSM997858,...,GSM997889,GSM997890,GSM997891,GSM997892,GSM997893,GSM997894,GSM997895,GSM997896,GSM997897,GSM997898
0,1007_s_at,474.82120,3764.07900,2193.57600,2712.1860,4876.2490,4297.08400,81.72649,4146.96500,3418.5030,...,4603.12700,2178.83300,5903.17100,888.87700,1480.285000,3165.808000,3920.06600,995.23600,1269.23500,2221.01000
1,1053_at,64.70827,78.76714,180.41580,424.7030,102.7654,307.33720,400.26320,135.27490,137.9451,...,449.11490,21.92653,366.58930,127.12930,265.340700,421.790700,510.07830,75.41043,164.05850,155.66470
2,117_at,202.39090,159.54120,139.19140,625.5750,696.8760,105.85680,390.57540,52.02566,477.9807,...,259.49240,171.23530,322.63720,19.56303,100.053800,299.769900,488.14100,42.98090,43.12577,67.33868
3,121_at,728.33920,253.09080,475.95720,539.3690,968.9207,467.66870,2066.34200,783.54440,1392.0420,...,739.03540,486.72830,657.47960,423.60690,364.608000,388.616100,507.78380,367.69270,391.64960,383.42200
4,1255_g_at,309.24050,342.40070,25.24212,117.4153,446.7377,28.12048,1490.31800,92.12761,419.9179,...,18.80592,110.53640,20.58882,18.15357,8.098538,6.897666,82.34903,46.26534,52.66693,32.73994


In [ ]:
gene_row_ids = genetic_data['ID'][:20].tolist()
gene_row_ids

['1007_s_at',
 '1053_at',
 '117_at',
 '121_at',
 '1255_g_at',
 '1294_at',
 '1316_at',
 '1320_at',
 '1405_i_at',
 '1431_at',
 '1438_at',
 '1487_at',
 '1494_f_at',
 '1552256_a_at',
 '1552257_a_at',
 '1552258_at',
 '1552261_at',
 '1552263_at',
 '1552264_a_at',
 '1552266_at']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [ ]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at', '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at', '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at', '1552263_at', '1552264_a_at', '1552266_at']\n"

If not required, jump directly to the gene normalization step

In [ ]:
requires_gene_mapping = True

In [ ]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)[0]
    gene_annotation_summary = preview_df(gene_annotation)
    gene_annotation_summary

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [ ]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = key_name1
    gene_name_key = key_name2

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = key_name1
    gene_name_key = key_name2

    Gene annotation dictionary:
    {'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Databa

In [ ]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'Gene Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of

In [ ]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 49 samples.


In [ ]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Sjögren's Syndrome', the least common label is '1.0' with 17 occurrences. This represents 34.69% of the dataset.
The distribution of the feature 'Sjögren's Syndrome' in this dataset is fine.



False

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [ ]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [5]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE84844'

In [6]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
1,GSE84844,True,True,False,True,False,60.0,
6,GSE143153,True,True,False,True,False,32.0,
8,GSE110007,True,True,False,True,False,29.0,


In [7]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [8]:
merged_data.head()

,Sjögren's Syndrome,Age,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A4GALT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0,45,5.739716,5.531918,4.073243,4.680171,8.261442,3.956358,5.001172,6.031274,...,5.349392,6.811687,4.856520,5.001800,7.012791,4.613131,7.855182,7.594642,6.093332,7.106518
1,0,40,5.452101,5.131475,4.106471,5.052298,9.217795,4.260219,4.639341,6.057871,...,5.193747,5.517230,5.310642,4.927082,7.025551,4.992692,8.006711,7.850989,6.596223,6.962768
2,0,54,5.784453,5.785170,3.978284,4.601635,8.902882,4.107508,4.809921,5.912525,...,5.696565,5.896517,5.604909,5.107001,6.872938,4.533009,8.112374,7.337760,6.405903,7.233815
3,0,50,5.283689,5.385669,4.178392,4.617995,8.780384,3.917632,5.144286,6.223132,...,5.053215,5.802501,5.288305,4.732636,7.052588,4.810001,8.001139,7.901643,6.893890,6.624012
4,0,31,5.414403,5.828662,3.836844,4.309026,8.221119,4.063579,4.561822,6.135777,...,5.269230,6.576080,5.135130,4.916972,6.706851,4.590855,7.819838,8.041268,6.623575,7.204644


In [9]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [10]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [11]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [12]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [14]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 50.00% ± 17.48%


In [15]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 86.67% ± 6.67%


In [16]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 5. Discussion and report

In [17]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.3684
Found 39 genes with non-zero coefficients associated with the trait 'Sjögren's Syndrome' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
    Variable  Coefficient  Absolute Coefficient
     DEFB127    -8.372770              8.372770
        RHOC     6.009638              6.009638
      RHBDF2     5.878851              5.878851
      RAB27B    -4.790219              4.790219
    CDC42BPA    -4.549917              4.549917
       ITIH1     4.536480              4.536480
        SELE     3.277567              3.277567
      ACTRT3     2.884623              2.884623
       P2RX3     2.527925              2.527925
       TMUB1     2.389405              2.389405
     ERVK3-2     2.312583              2.312583
       GSK3B    -2.208369              2.208369
        GFAP     2.068015              2.068015
        TPBG    -1.852632              1.852632
   

/content/drive/MyDrive/Colab Notebooks/utils.py:434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [18]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.3684
Found 0 significant genes associated with the trait 'Sjögren's Syndrome' conditional on the factor 'Age', with corrected p-value < 0.05:
Empty DataFrame
Columns: [Variable, Coefficient, p_value, corrected_p_value]
Index: []


/content/drive/MyDrive/Colab Notebooks/utils.py:443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
